In [11]:
import struct as st
import numpy as np

from net import *
from net.util import relu, d_relu, cross_entropy

In [12]:
# Helper function to parse IDX files
def parse_idx(file_path):
    with open(file_path, 'rb') as file:
        magic = st.unpack('>I', file.read(4))[0]  # Magic number (4 bytes)
        num_items = st.unpack('>I', file.read(4))[0]  # Number of items (4 bytes)

        if magic == 2051:  # Magic number for images
            num_rows = st.unpack('>I', file.read(4))[0]
            num_cols = st.unpack('>I', file.read(4))[0]
            num_bytes = num_items * num_rows * num_cols
            data = np.frombuffer(file.read(num_bytes), dtype=np.uint8)
            return data.reshape(num_items, num_rows, num_cols)
        elif magic == 2049:  # Magic number for labels
            data = np.frombuffer(file.read(num_items), dtype=np.uint8)
            return data
        else:
            raise ValueError(f"Unknown magic number: {magic}")

# Parse the training data
x_train = parse_idx('../data/DigitData/train-images.idx3-ubyte')
y_train = parse_idx('../data/DigitData/train-labels.idx1-ubyte')

x_test = parse_idx('../data/DigitData/t10k-images.idx3-ubyte')
y_test = parse_idx('../data/DigitData/t10k-labels.idx1-ubyte')

# Reshape and scale down
p = x_train.shape[1]
x_train = x_train.reshape(x_train.shape[0], -1) / 255.0
x_test = x_test.reshape(x_test.shape[0], -1) / 255.0
print(p)

28


In [13]:
# Normalize
train_mean = np.mean(x_train, axis=0)
train_std = np.std(x_train, axis=0) + 1e-12

x_train = (x_train - train_mean) / train_std
x_test = (x_test - train_mean) / train_std

x_train = x_train.reshape(x_train.shape[0], 1, p, p) # Need (n, c, p, p) shape
x_test = x_test.reshape(x_test.shape[0], 1, p, p)

In [14]:
# Build Network -- Values are Hyperparameters
strides = 2
c1 = 8
size_k1 = 3
size_p1 = 2
c2 = 16
size_k2 = 3
size_p2 = 2
size_d1 = 128
k1 = Tensor(np.random.randn(c1, 1, size_k1, size_k1) * 0.01)
C1 = Convolutional(k1)
B1 = BatchNorm(c1)
A1 = Activation(relu, d_relu)
P1 = Pool(size_p1, stride=strides)

k2= Tensor(np.random.randn(c2, c1, size_k2, size_k2) * 0.01)
C2 = Convolutional(k2)
B2 = BatchNorm(c2)
A2 = Activation(relu, d_relu)
P2 = Pool(size_p2, stride=strides)
F = Flatten()
inp_d1 = 16 * 7 * 7  # Computed based on output dims after 2 conv+pool layers
D1 = Dense(inp_d1, size_d1)
A3 = Activation(relu, d_relu)
D2 = Dense(size_d1, 10)

CNN = Network([C1, B1, A1, P1, C2, B2, A2, P2, F, D1, A3, D2])

In [15]:
# Hyper parameters
LR = 0.1
STEPS = 5000
BATCH_SIZE = 30

In [16]:
# Train
grad_descent(CNN, cross_entropy, x_train, y_train, STEPS, BATCH_SIZE, LR)

Loss at Step: 1: 3.2634399321654444
Loss at Step: 101: 0.25796681032435137
Loss at Step: 201: 0.35970319081459984
Loss at Step: 301: 0.04736156376934284
Loss at Step: 401: 0.3781280805437729
Loss at Step: 501: 0.10142891819750324
Loss at Step: 601: 0.09987757882653142
Loss at Step: 701: 0.0776410733430767
Loss at Step: 801: 0.00915989446325586
Loss at Step: 901: 0.041694478047796364
Loss at Step: 1001: 0.07270260066600613
Loss at Step: 1101: 0.11846535919759993
Loss at Step: 1201: 0.07396307217327408
Loss at Step: 1301: 0.24777959661423016
Loss at Step: 1401: 0.2695538268265702
Loss at Step: 1501: 0.006221386268717489
Loss at Step: 1601: 0.20986911709390269
Loss at Step: 1701: 0.050403183035608284
Loss at Step: 1801: 0.23386978052059051
Loss at Step: 1901: 0.020516588106240817
Loss at Step: 2001: 0.038786329530170235
Loss at Step: 2101: 0.0252266773520183
Loss at Step: 2201: 0.03935032581351981
Loss at Step: 2301: 0.12112966665938461
Loss at Step: 2401: 0.01780252668212791
Loss at Step

In [17]:
# Training Loss
CNN.set_to_predict()
logits = CNN.forward(x_test[:8000])
print(cross_entropy(logits.value, y_test[:8000], grad=False))

0.05839596207484694


In [18]:
preds = np.argmax(logits.value, axis=1)
print(np.mean(preds == y_test[:8000]))

0.98075
